In [1]:
import os
import sys
import numpy as np 
import open3d as o3d
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import collections
import tensorflow_datasets as tfds

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


2024-07-04 08:50:57.766308: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-04 08:50:58.132847: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-04 08:50:58.135307: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-04 08:50:59.193368: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df = pd.read_pickle("../final_dataset.pkl") 
df.reset_index(inplace=True, drop=True)
df.head()

,id,voxelgrids,positions,poses,odoms
0,-1,"[[[25, 53], [32, 2], [17, 54], [26, 50], [30, ...","[[0.7827229554855963, -0.6603078958156333, 4.5...","[[[31.0, 57.0, 0.1766081601381302], [27.0, 52....","[[0.00022100093561436428, -0.00164606522993054..."
1,-1,"[[[17, 38], [36, 71], [18, 39], [29, 53], [24,...","[[1.8763295554092785, -0.7392371475816817, 8.6...","[[[17.0, 1.0, 0.22867129743099213], [17.0, 1.0...","[[-0.0010584275078791316, 0.005180639174413457..."
2,-1,"[[[13, 56], [8, 46], [20, 66], [44, 3], [43, 6...","[[1.2298735958886313, -0.6599135136166747, 8.2...","[[[19.0, 13.0, 0.1444173902273178], [17.0, 11....","[[-0.017197470816407222, -0.008501126993886861..."
3,-1,"[[[29, 68], [66, 59], [11, 93], [73, 58], [24,...","[[0.49709940835786537, -0.62631209064576, 7.09...","[[[18.0, 5.0, 0.17500989139080048], [21.0, 2.0...","[[0.00810160200217194, -0.006334385323650088, ..."
4,-1,"[[[22, 6], [23, 14], [23, 15], [26, 21], [23, ...","[[0.9673029278573837, -0.3436083278432631, 2.9...","[[[62.0, 62.0, 0.14962413907051086], [73.0, 45...","[[-0.027349297901359395, -0.011966854241542754..."


In [3]:
num_steps = 15

In [4]:
position_list = []
pose_list = []
for index, row in df.iterrows():
    if row['positions'].shape[0] >= num_steps:
        pos_data = row['positions']
        poses_data = row['poses'].reshape(-1,17*3)
        vox_data = row['voxelgrids'].reshape(-1,1000*2)
        #data_df = tf.data.Dataset.from_tensor_slices({"input_pos": pos_data, "input_poses":poses_data,"input_grid": vox_data})
        #print(data_df)
        pos_df = tf.data.Dataset.from_tensor_slices(pos_data)
        pose_df = tf.data.Dataset.from_tensor_slices(poses_data)
        pos_df = pos_df.window(num_steps, shift=3, drop_remainder=True)
        pose_df = pose_df.window(num_steps, shift=3, drop_remainder=True)
        for windows1, windows2 in zip(pos_df, pose_df):
            np_arr1 = np.asarray([item.numpy() for item in windows1])
            #print(np_arr1.shape)
            position_list.append(np_arr1)

            np_arr2 = np.asarray([item.numpy() for item in windows2])
            #print(np_arr2.shape)
            pose_list.append(np_arr2)

            #dataset_list.append(tf.data.Dataset.from_tensors(windows))
            

2024-07-04 08:51:45.366624: W tensorflow/core/framework/dataset.cc:956] Input of Window will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.
2024-07-04 08:52:15.369800: W tensorflow/core/framework/dataset.cc:956] Input of Window will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


In [5]:
pos_dataset = tf.data.Dataset.from_tensor_slices(np.asarray(position_list))
pose_dataset = tf.data.Dataset.from_tensor_slices(np.asarray(pose_list))
zip_ds = tf.data.Dataset.zip((pos_dataset, pose_dataset))
zip_ds = zip_ds.shuffle(buffer_size=100, reshuffle_each_iteration=True)
zip_ds = zip_ds.batch(32).prefetch(tf.data.experimental.AUTOTUNE)

In [6]:
for data1, data2 in zip_ds:
    print(data1.shape, data2.shape)
    break

(32, 15, 3) (32, 15, 51)


In [7]:
len(zip_ds)

87

In [8]:
# shuffle, batch and split dataset
DATASET_SIZE = len(zip_ds)

train_size = int(0.8 * DATASET_SIZE)
test_size = int(0.2 * DATASET_SIZE)

train_dataset = zip_ds.take(train_size)
test_dataset = zip_ds.skip(train_size)

In [9]:
len(train_dataset)
# 26 batches for len 20 and shift 5
# 41 batches for len 20 and shift 3
# 60 batches for len 15 and shift 3


69

In [14]:
for x in train_dataset:
    print(x[1])
    break

tf.Tensor(
[[[2.60000000e+01 3.10000000e+01 2.90879875e-01 ... 3.80000000e+01
   1.60000000e+02 4.20909047e-01]
  [2.50000000e+01 4.20000000e+01 8.85042250e-02 ... 3.90000000e+01
   1.64000000e+02 3.86540353e-01]
  [2.90000000e+01 1.80000000e+01 2.38930687e-01 ... 3.40000000e+01
   1.15000000e+02 6.04797006e-01]
  ...
  [2.00000000e+01 1.20000000e+01 3.18613201e-01 ... 2.90000000e+01
   1.29000000e+02 6.68862522e-01]
  [2.00000000e+01 9.00000000e+00 2.71391094e-01 ... 2.80000000e+01
   1.24000000e+02 4.84991878e-01]
  [2.10000000e+01 1.30000000e+01 2.11803406e-01 ... 2.20000000e+01
   1.19000000e+02 4.10803765e-01]]

 [[2.90000000e+01 2.20000000e+01 2.21498668e-01 ... 3.50000000e+01
   1.83000000e+02 5.08365989e-01]
  [2.80000000e+01 2.30000000e+01 2.95080930e-01 ... 3.30000000e+01
   1.82000000e+02 4.67873484e-01]
  [3.60000000e+01 2.00000000e+01 2.60397196e-01 ... 3.40000000e+01
   1.89000000e+02 4.16139275e-01]
  ...
  [5.00000000e+01 1.70000000e+01 2.74160713e-01 ... 4.70000000e+01

In [10]:
train_ds_x1 = train_dataset.map(lambda x,y: x)
train_ds_x2 = train_dataset.map(lambda x, y: y)

val_ds_x1 = test_dataset.map(lambda x,y: x)
val_ds_x2 = test_dataset.map(lambda x, y: y)

In [10]:
# create mask array, True = needs to be predicted
# TODO: adapt to currrent num_steps
mask_arrays = []
for i in range(len(train_dataset)):
    new_batch = []
    for j in range(20):
        mask_arr = [False] * 10 + [True] *10
        # hide 0-3 in between steps (for lazyness whole datapoint)
        hidden_nr = np.random.randint(4)
        hidden_idx = np.random.choice(range(10),hidden_nr, replace=False)
        for v in hidden_idx:
            mask_arr[v] = True
        new_batch.append(mask_arr)
    mask_arrays.append(new_batch)
# transform to match with batch and training, test and val datasets

# Data augmentation

https://github.com/google-research/human-scene-transformer/blob/main/human_scene_transformer/jrdb/input_fn.py

# Test single layers

In [11]:
""" Adapted Sinusoidal Embedding Layer from source: https://github.com/google-research/human-scene-transformer/blob/main/human_scene_transformer/model/embedding.py    """
class SinusoidalEmbeddingLayer(tf.keras.layers.Layer):
  """Sinusoidal Postional Embedding for xyz and time."""

  def __init__(self, min_freq=4, max_freq=256, hidden_size=256):
    super().__init__()
    self.min_freq = float(min_freq)
    self.max_freq = float(max_freq)
    self.hidden_size = hidden_size
    if hidden_size % 2 != 0:
      raise ValueError('hidden_size ({hidden_size}) must be divisible by 2.')
    self.num_freqs_int32 = hidden_size // 2
    self.num_freqs = tf.cast(self.num_freqs_int32, dtype=tf.float32)

  def build(self, input_shape):
    log_freq_increment = (
        tf.math.log(float(self.max_freq) / float(self.min_freq)) /
        tf.maximum(1.0, self.num_freqs - 1))
    # [num_freqs]
    self.inv_freqs = self.min_freq * tf.exp(
        tf.range(self.num_freqs, dtype=tf.float32) * -log_freq_increment)

  def call(self, input_tensor):
    
    # [..., num_freqs]
    input_tensor = tf.expand_dims(input_tensor, -1)
    input_tensor = tf.tile(input_tensor, [1, 1, 1, self.num_freqs_int32])
    
    # Compute the sinusoidal embeddings
    sin_embeds = tf.sin(input_tensor * self.inv_freqs)
    cos_embeds = tf.cos(input_tensor * self.inv_freqs)
    
    # Concatenate along the last axis
    embedded = tf.concat([sin_embeds, cos_embeds], axis=-1)
    
    # Reshape to the desired output shape (batch_size, sequence_length, feature_size * hidden_size)
    batch_size = tf.shape(embedded)[0]
    sequence_length = tf.shape(embedded)[1]
    feature_size = tf.shape(embedded)[2]
    
    embedded = tf.reshape(embedded, (batch_size, sequence_length, feature_size * self.hidden_size))
    """
    # [..., num_freqs]
    input_tensor = tf.repeat(
        input_tensor[..., tf.newaxis], self.num_freqs_int32, axis=-1)
    # [..., h]
    embedded = tf.concat([
        tf.sin(input_tensor * self.inv_freqs),
        tf.cos(input_tensor * self.inv_freqs)
    ],
                         axis=-1)"""
    return embedded

In [12]:
""" Adapted Agent Position Encoding Layer from source: https://github.com/google-research/human-scene-transformer/blob/main/human_scene_transformer/model/agent_feature_encoder.py    """
class AgentPositionEncoder(tf.keras.layers.Layer):
  """Encodes agents spatial positions."""

  def __init__(self, output_shape, embedding_size):
    
    super().__init__()

    self.embedding_layer = SinusoidalEmbeddingLayer(
        hidden_size=embedding_size) # output_shape (batch_sie, sequence_length, feature size, hidden_size)

    self.mlp = tf.keras.layers.EinsumDense(
        '...f,fh->...h',
        output_shape=output_shape,
        bias_axes='h',
        
        activation=None)

  def call(self, input_batch):
    return self.mlp(self.embedding_layer(input_batch[0]))

In [13]:
""" Adapted Agent Keypoint Encoding Layer from source: https://github.com/google-research/human-scene-transformer/blob/main/human_scene_transformer/model/agent_feature_encoder.py    """
class AgentKeypointsEncoder(tf.keras.layers.Layer):
  """Encodes the agent's keypoints."""

  def __init__(self, output_shape, embedding_size):
    super().__init__()

    self.mlp1 = tf.keras.layers.EinsumDense(
        '...f,fh->...h',
        output_shape=output_shape,
        bias_axes='h',
        activation=tf.nn.relu)

  def call(self, input_batch, training=None):

    keypoints = input_batch[1]

    out = self.mlp1(keypoints)

    return out

In [14]:
from typing import Tuple, Optional
class PreprocessLayer(tf.keras.layers.Layer):
    """ Applies the masking to the sequence
    """

    def calc_hidden_mask(self, dataset):
        # create mask array, False = needs to be predicted
        sequence_length = 15
        mask_arrays = []
        mask_arr = [True] * 6 + [False] * 9
        # hide 0-2 in between steps (for lazyness whole datapoint)
        hidden_nr = np.random.randint(3)
        hidden_idx = np.random.choice(range(6),hidden_nr, replace=False)
        for v in hidden_idx:
            mask_arr[v] = False
        print(np.asarray(mask_arr).shape)
        return np.asarray(mask_arr)

    def call(self,
           raw_input_batch: Tuple[tf.Tensor, tf.Tensor],
           is_hidden: Optional[tf.Tensor] = None) -> Tuple[Tuple[tf.Tensor, tf.Tensor], tf.Tensor]:
        input_batch = raw_input_batch
        mask = self.calc_hidden_mask(input_batch) #tf.convert_to_tensor
        mask_tensor = tf.constant(mask, dtype=tf.bool)

        batch_size = tf.shape(input_batch[0])[0]
        sequence_length = tf.shape(input_batch[0])[1]
        feature_size1 = tf.shape(input_batch[0])[2]
        feature_size2 = tf.shape(input_batch[1])[2]

        # Expand dimensions of mask to match the input tensor
        expanded_mask = tf.expand_dims(mask_tensor, axis=0)  # Add batch dimension
        expanded_mask = tf.expand_dims(expanded_mask, axis=-1)  # Add feature dimension

        # Broadcast mask to match input tensor shape
        broadcasted_mask_pos = tf.broadcast_to(expanded_mask, (batch_size, sequence_length, feature_size1))
        broadcasted_mask_pose = tf.broadcast_to(expanded_mask, (batch_size, sequence_length, feature_size2))

        #batch_mask = tf.broadcast_to(expanded_mask, (batch_size, sequence_length))

        # Apply mask
        masked_input_pos = tf.where(broadcasted_mask_pos, input_batch[0], tf.zeros_like(input_batch[0]))
        masked_input_pose = tf.where(broadcasted_mask_pose, input_batch[1], tf.zeros_like(input_batch[1]))

        return (masked_input_pos, masked_input_pose), mask

In [15]:
class AgentTemporalEncoder(tf.keras.layers.Layer):
  """Encodes agents temporal positions."""

  def __init__(self,output_shape, embedding_size, num_steps):
    super().__init__()
    self.embedding_layer = SinusoidalEmbeddingLayer(
        max_freq=num_steps,
        hidden_size=embedding_size)

    self.mlp = tf.keras.layers.EinsumDense(
        '...f,fh->...h',
        output_shape=output_shape,
        bias_axes='h',
        activation=None)

  def _get_temporal_embedding(self, input_batch):
    # This weird thing is for exporting and loading keras model...
    b = tf.shape(input_batch[0])[0]
    num_steps = tf.shape(input_batch[0])[1]

    t = tf.range(0, num_steps, dtype=tf.float32)
    t = t[tf.newaxis, :]
    t = tf.tile(t, [b, 1])
    return self.embedding_layer(t[..., tf.newaxis])

  def call(self, input_batch):
    return self.mlp(self._get_temporal_embedding(input_batch))

In [16]:
class FeatureAttnAgentEncoderLearnedLayer(tf.keras.layers.Layer):
  """Independently encodes features and attends to them.

  Agent features are cross-attended with a learned query or hidden_vecs instead
  of MLP.
  """

  def __init__(self, input_length, batch_size=32, hidden_size=128, num_heads=4, ln_eps=1e-6, transformer_ff_dim=128, drop_prob=0.1):
    super(FeatureAttnAgentEncoderLearnedLayer, self).__init__()

    self.batch_size=batch_size
    self.input_length = input_length
    self.num_heads=num_heads

    # Cross Attention and learned query.
    self.attn_layer = tf.keras.layers.MultiHeadAttention(
        num_heads=num_heads,
        key_dim=360//num_heads,  # "large" to prevent a bottleneck
        value_dim=360//num_heads)
    self.attn_ln = tf.keras.layers.LayerNormalization(epsilon=ln_eps)
    self.ff_layer1 = tf.keras.layers.EinsumDense(
        '...h,hf->...f',
        output_shape=transformer_ff_dim,
        bias_axes='f',
        activation='relu')
    self.ff_layer2 = tf.keras.layers.EinsumDense(
        '...f,fh->...h',
        output_shape=hidden_size,
        bias_axes='h',
        activation=None)
    self.ff_dropout = tf.keras.layers.Dropout(drop_prob)
    self.ff_ln = tf.keras.layers.LayerNormalization(epsilon=ln_eps)

    self.agent_feature_embedding_layers = []
    # Position Feature
    self.agent_feature_embedding_layers.append(
        AgentPositionEncoder(output_shape=hidden_size-8, embedding_size=hidden_size))
    # Feature Embedding - keypoints
    self.agent_feature_embedding_layers.append(
        AgentKeypointsEncoder(output_shape=hidden_size-8, embedding_size=hidden_size))

    # Temporal Embedding
    self.agent_feature_embedding_layers.append(
        AgentTemporalEncoder(output_shape=hidden_size-8, embedding_size=hidden_size, num_steps=input_length))

    # [1, 1, h]
    self.learned_query_vec = tf.Variable(
        tf.random_uniform_initializer(
            minval=-1., maxval=1.)(shape=[1, 1, hidden_size]),
        trainable=True,
        dtype=tf.float32)

  def _build_learned_query(self, input_batch):
    """Converts self.learned_query_vec into a learned query vector."""
    # This weird thing is for exporting and loading keras model...
    b = tf.shape(input_batch[0])[0]
    num_steps = tf.shape(input_batch[0])[1]

    # [b, num_steps, h]
    return tf.tile(self.learned_query_vec, [b, num_steps, 1])

  def call(self, input_batch: Tuple[Tuple[tf.Tensor, tf.Tensor], tf.Tensor],
           training: Optional[bool] = None):
    mask = input_batch[1]
    input_batch = input_batch[0]
    layer_embeddings = []
    for layer in self.agent_feature_embedding_layers:
      layer_embedding = layer(input_batch, training=training)
      layer_embeddings.append(layer_embedding)
    embedding = tf.concat(layer_embeddings, axis=-1)

    b = tf.shape(embedding)[0]
    t = tf.shape(embedding)[1]
    n = tf.shape(embedding)[2]

    #print("embedding shape: ", embedding.shape)

    attention_mask = tf.where(mask, 1, 0)
    attention_mask = tf.cast(attention_mask, dtype=tf.float32)
    attention_mask = tf.expand_dims(attention_mask, axis=0)
    batch_size = tf.shape(input_batch[0])[0]
    attention_mask = tf.broadcast_to(attention_mask, (batch_size, self.input_length, self.input_length))
    attention_mask = tf.expand_dims(attention_mask, axis=1)  # Shape: (batch_size, 1, seq_len, seq_len)
    attention_mask = tf.repeat(attention_mask, self.num_heads, axis=1)  # Shape: (batch_size, num_heads, seq_len, seq_len)
    #print("attention_mask shape: ", attention_mask.shape)
    
    learned_query = self._build_learned_query(input_batch)
    #print("learned_query shape: ", learned_query.shape)

    # Attention along axis 3
    attn_out, attn_score = self.attn_layer(
        query=learned_query,
        key=embedding,
        value=embedding,
        attention_mask=attention_mask,
        return_attention_scores=True)
    #print("attn_out shape: ", attn_out.shape)
    # [b, t, h]
    #attn_out = attn_out[..., 0, :]
    out = self.ff_layer1(attn_out)
    out = self.ff_layer2(out)
    out = self.ff_dropout(out, training=training)
    out = self.ff_ln(out + attn_out)

    return out, attn_score

In [17]:
class AgentSelfAlignmentLayer(tf.keras.layers.Layer):
  """Enables agent to become aware of its temporal identity.

  Agent features are cross-attended with a learned query in temporal dimension.
  """

  def __init__(self,
               num_heads=8,
               hidden_size=128,
               ln_eps=1e-6,
               ff_dim=128):
    super().__init__()
    self.hidden_size = hidden_size
    self.num_heads=num_heads
    if hidden_size % num_heads != 0:
      raise ValueError(f'hidden_size ({hidden_size}) must be an integer '
                       f'times bigger than num_heads ({num_heads}).')
    self.attn_layer = tf.keras.layers.MultiHeadAttention(
        num_heads=num_heads,
        key_dim=ff_dim // num_heads,
        value_dim = ff_dim // num_heads,
        attention_axes=1)
    self.attn_ln = tf.keras.layers.LayerNormalization(epsilon=ln_eps)
    self.ff_layer1 = tf.keras.layers.EinsumDense(
        '...h,hf->...f', output_shape=ff_dim, bias_axes='f', activation='relu')
    self.ff_layer2 = tf.keras.layers.EinsumDense(
        '...f,fh->...h',
        output_shape=hidden_size,
        bias_axes='h',
        activation=None)
    self.ff_ln = tf.keras.layers.LayerNormalization(epsilon=ln_eps)

    # [1, 1, h]
    self.learned_query_vec = tf.Variable(
        tf.random_uniform_initializer(
            minval=-1., maxval=1.)(shape=[1, 1, hidden_size]),
        trainable=True,
        dtype=tf.float32)

  def build_learned_query(self, input_batch):
    """Converts self.learned_query_vec into a learned query vector."""
    # This weird thing is for exporting and loading keras model...
    b = tf.shape(input_batch)[0]
    t = tf.shape(input_batch)[1]

    # [b, t, 1, h]
    return tf.tile(self.learned_query_vec, [b, t, 1])

  def call(self, input_batch):
    # [b, t, h]
    hidden_vecs = input_batch[0]
    mask = input_batch[1]

    # Expand the attention mask with new dims so that Keras can broadcast to
    # the same shape as the attn_score: [b, num_heads, a, t, a, t].
    # attn_mask shape: [b, 1, 1, 1 t,]
    # True means the position participate in the attention while all
    # False positions are ignored.
    
    #print("input batch shape: ", hidden_vecs.shape)
    
    attention_mask = tf.where(mask, 1, 0)
    attention_mask = tf.cast(attention_mask, dtype=tf.float32)
    attention_mask = tf.expand_dims(attention_mask, axis=0)
    batch_size = tf.shape(hidden_vecs)[0]
    input_length = tf.shape(hidden_vecs)[1]
    attention_mask = tf.broadcast_to(attention_mask, (batch_size, input_length, input_length))
    attention_mask = tf.expand_dims(attention_mask, axis=1)  # Shape: (batch_size, 1, seq_len, seq_len)
    attention_mask = tf.repeat(attention_mask, self.num_heads, axis=1)  # Shape: (batch_size, num_heads, seq_len, seq_len)

    # [b, t, 1, h]
    learned_query = self.build_learned_query(hidden_vecs)
    attn_out, attn_score = self.attn_layer(
        query=learned_query,
        key=hidden_vecs,
        value=hidden_vecs,
        attention_mask=attention_mask,
        return_attention_scores=True)

    attn_out = self.attn_ln(attn_out + hidden_vecs)

    # Feed-forward layers.
    out = self.ff_layer1(attn_out)
    out = self.ff_layer2(out)
    out = self.ff_ln(out + attn_out)


    return out, attn_score

In [29]:
class SelfAttnTransformerLayer(tf.keras.layers.Layer):
  """Performs full self-attention across the agent and time dimensions."""

  def __init__(
      self,
      num_heads=8,
      hidden_size=128,
      drop_prob=0.1,
      ln_eps=1e-6,
      ff_dim=128,
      mask=False,
      flatten=False,
      multimodality_induced=False,
  ):
    super().__init__()
    self.hidden_size = hidden_size
    self.mask = mask
    self.flatten = flatten
    self.multimodality_induced = multimodality_induced
    if hidden_size % num_heads != 0:
      raise ValueError(
          f'hidden_size ({hidden_size}) must be an integer '
          f'times bigger than num_heads ({num_heads}).'
      )
    self.attn_layer = tf.keras.layers.MultiHeadAttention(
        num_heads=num_heads,
        key_dim=hidden_size // num_heads,
        attention_axes=1,
    )  # Full Attention time
    self.attn_dropout = tf.keras.layers.Dropout(drop_prob)
    self.attn_ln = tf.keras.layers.LayerNormalization(epsilon=ln_eps)
    self.ff_layer1 = tf.keras.layers.EinsumDense(
        '...h,hf->...f', output_shape=ff_dim, bias_axes='f', activation='relu'
    )
    self.ff_layer2 = tf.keras.layers.EinsumDense(
        '...f,fh->...h',
        output_shape=hidden_size,
        bias_axes='h',
        activation=None,
    )
    self.ff_dropout = tf.keras.layers.Dropout(drop_prob)
    self.ff_ln = tf.keras.layers.LayerNormalization(epsilon=ln_eps)

  def call(self, input_batch, training=None):
    # [b, t, h] or [b, t, n, h]
    hidden_vecs = input_batch[0]
    mask = input_batch[1]

    if self.flatten:
      h_shape = tf.shape(hidden_vecs)
      b = h_shape[0]
      t = h_shape[1]
      h = h_shape[-1]

      if self.multimodality_induced:
        n = h_shape[3]
        hidden_vecs = tf.reshape(hidden_vecs, (b, -1, n, h))
      else:
        hidden_vecs = tf.reshape(hidden_vecs, (b, -1, h))

    # Expand the attention mask with new dims so that Keras can broadcast to
    # the same shape as the attn_score: [b, num_heads, a, t, a, t].
    # attn_mask shape: [b, 1, 1, 1, a, t,]
    # True means the position participate in the attention while all
    # False positions are ignored.
    if not self.mask:
      attn_mask = None
    else:
      attention_mask = tf.where(mask, 1, 0)
      attention_mask = tf.cast(attention_mask, dtype=tf.float32)
      attention_mask = tf.expand_dims(attention_mask, axis=0)
      batch_size = tf.shape(hidden_vecs)[0]
      input_length = tf.shape(hidden_vecs)[1]
      attention_mask = tf.broadcast_to(attention_mask, (batch_size, input_length, input_length))
      attn_mask = tf.expand_dims(attention_mask, axis=1)  # Shape: (batch_size, 1, seq_len, seq_len)

    if self.multimodality_induced:  # We have modes
      attn_mask = tf.expand_dims(attn_mask, axis=1)  # Shape: (batch_size, 1, 1, seq_len, seq_len)


    attn_out, attn_score = self.attn_layer(
        query=hidden_vecs,
        key=hidden_vecs,
        value=hidden_vecs,
        attention_mask=attn_mask,
        return_attention_scores=True)
    out = self.attn_dropout(attn_out, training=training)
    attn_out = self.attn_ln(out + hidden_vecs)

    # Feed-forward layers.
    out = self.ff_layer1(attn_out)
    out = self.ff_layer2(out)
    out = self.ff_dropout(out, training=training)
    out = self.ff_ln(out + attn_out)

    if self.flatten:
      out = tf.reshape(out, h_shape)

    return out, attn_score

In [28]:
class SelfAttnModeTransformerLayer(tf.keras.layers.Layer):
  """Performs full self-attention across the future modes dimensions."""

  def __init__(self,
               num_heads=8,
               hidden_size=128,
               drop_prob=0.1,
               ln_eps=1e-6,
               ff_dim=128):
    super().__init__()
    self.hidden_size = hidden_size
    if hidden_size % num_heads != 0:
      raise ValueError(f'hidden_size ({hidden_size}) must be an integer '
                       f'times bigger than num_heads ({num_heads}).')
    self.attn_layer = tf.keras.layers.MultiHeadAttention(
        num_heads=num_heads,
        key_dim=hidden_size // num_heads,
        attention_axes=2)  # Attention over modes
    self.attn_dropout = tf.keras.layers.Dropout(drop_prob)
    self.attn_ln = tf.keras.layers.LayerNormalization(epsilon=ln_eps)
    self.ff_layer1 = tf.keras.layers.EinsumDense(
        '...h,hf->...f', output_shape=ff_dim, bias_axes='f', activation='relu')
    self.ff_layer2 = tf.keras.layers.EinsumDense(
        '...f,fh->...h',
        output_shape=hidden_size,
        bias_axes='h',
        activation=None)
    self.ff_dropout = tf.keras.layers.Dropout(drop_prob)
    self.ff_ln = tf.keras.layers.LayerNormalization(epsilon=ln_eps)

  def call(self, input_batch, training=None):

    # [b, t, n, h]
    hidden_vecs = input_batch

    attn_out, attn_score = self.attn_layer(
        query=hidden_vecs,
        key=hidden_vecs,
        value=hidden_vecs,
        attention_mask=None,
        return_attention_scores=True)
    out = self.attn_dropout(attn_out, training=training)
    attn_out = self.attn_ln(out + hidden_vecs)

    # Feed-forward layers.
    out = self.ff_layer1(attn_out)
    out = self.ff_layer2(out)
    out = self.ff_dropout(out, training=training)
    out = self.ff_ln(out + attn_out)

    return out, attn_score

In [27]:
class MultimodalityInduction(tf.keras.layers.Layer):
  """Enables the model to forward and predict multi-mode predictions.

  1) Features are broadcasted to number of modes and summed with learned mode
      tensors.
  2) Mixture Weights are generated by cross-attention over all dimensions
      between learned mode tensors and hidden tensors.
  """

  def __init__(self,
               num_modes=5,
               num_heads=8,
               hidden_size=128,
               drop_prob=0.1,
               ln_eps=1e-6,
               ff_dim=128):
    super().__init__()
    self.num_modes = num_modes
    self.hidden_size = hidden_size
    if hidden_size % num_heads != 0:
      raise ValueError(f'hidden_size ({hidden_size}) must be an integer '
                       f'times bigger than num_heads ({num_heads}).')
    self.mm_attn_layer = tf.keras.layers.MultiHeadAttention(
        num_heads=num_heads,
        key_dim=hidden_size // num_heads,
        attention_axes=3)
    self.mm_attn_ln = tf.keras.layers.LayerNormalization(epsilon=ln_eps)
    self.mm_ff_layer1 = tf.keras.layers.EinsumDense(
        '...h,hf->...f', output_shape=ff_dim, bias_axes='f', activation='relu')
    self.mm_ff_layer2 = tf.keras.layers.EinsumDense(
        '...f,fh->...h',
        output_shape=hidden_size,
        bias_axes='h',
        activation=None)
    self.mm_ff_ln = tf.keras.layers.LayerNormalization(epsilon=ln_eps)

    self.mw_attn_layer = tf.keras.layers.MultiHeadAttention(
        num_heads=num_heads,
        key_dim=hidden_size // num_heads,
        attention_axes=None)
    self.mw_attn_ln = tf.keras.layers.LayerNormalization(epsilon=ln_eps)
    self.mw_ff_layer1 = tf.keras.layers.EinsumDense(
        '...h,hf->...f', output_shape=ff_dim, bias_axes='f', activation='relu')
    self.mw_ff_layer2 = tf.keras.layers.EinsumDense(
        '...f,fh->...h',
        output_shape=1,  # Single logit per mode
        bias_axes='h',
        activation=None)
    self.mw_ff_ln = tf.keras.layers.LayerNormalization(epsilon=ln_eps)

    self.attn_dropout = tf.keras.layers.Dropout(drop_prob)
    self.ff_dropout = tf.keras.layers.Dropout(drop_prob)

    # [1, 1, m, h]
    self.learned_add_mm = tf.Variable(
        tf.random_uniform_initializer(
            minval=-1.,
            maxval=1.)(shape=[1, 1, self.num_modes, hidden_size]),
        trainable=True,
        dtype=tf.float32)

  def call(self, input_batch, training=None):
    mask = input_batch[1]
    # [b, t, 1, h]
    hidden_vecs = input_batch[0][..., tf.newaxis, :]

    # Multi Modes
    mm_add = self.mm_attn_ln(self.learned_add_mm + hidden_vecs)

    # Feed-forward layers.
    out = self.mm_ff_layer1(mm_add)
    out = self.mm_ff_layer2(out)
    out = self.ff_dropout(out)
    out = self.mm_ff_ln(out + hidden_vecs)

    # Mixture Weights
    # [b, 1, n, h]
    b = tf.shape(out)[0]
    attn_out_mw = self.mw_attn_layer(
        query=tf.tile(self.learned_add_mm, [b, 1, 1, 1]),
        key=mm_add,
        value=mm_add,
        return_attention_scores=False)
    attn_out_mw = self.attn_dropout(attn_out_mw, training=training)

    # [b, 1, 1, n, h]
    attn_out_mw = self.mw_attn_ln(attn_out_mw)

    # Feed-forward layers.
    out_mw = self.mw_ff_layer1(attn_out_mw)
    out_mw = self.mw_ff_layer2(out_mw)
    out_mw = self.ff_dropout(out_mw, training=training)

    # [b, 1, n]
    mixture_logits = out_mw[..., 0]
    return out, mixture_logits

In [38]:
# Adapted from 2D Predictions to 3D predictions
class Prediction2DPositionHeadLayer(tf.keras.layers.Layer):
  """Converts transformer hidden vectors to model predictions."""

  def __init__(self, hidden_units=None, num_stages=5):
    super().__init__()

    self.dense_layers = []
    # Add hidden layers.
    if hidden_units is not None:
      for units in hidden_units:
        self.dense_layers.append(
            tf.keras.layers.Dense(units, activation='relu'))
    # Add the final prediction head.
    self.dense_layers.append(
        tf.keras.layers.EinsumDense(
            '...h,hf->...f',
            output_shape=5,
            bias_axes='f',
            activation=None))

  def call(self, input_batch):
    # [b, t, n,  h]
    hidden_vecs = input_batch

    x = hidden_vecs
    # [b, t, n, 5]
    for layer in self.dense_layers:
      x = layer(x)
    pred = x
    """predictions = {
        'agents/position': pred[..., 0:3],
        'agents/position/raw_scale_tril': pred[..., 3:5],
    }"""
    return pred

In [36]:

input_length=15
input_dim_1 = 3
input_dim_2 = 51

# Define a simple model that includes the custom layer
input_1 = tf.keras.Input(shape=(input_length, input_dim_1))
input_2 = tf.keras.Input(shape=(input_length, input_dim_2))

hidden_size = 128
masked_inputs, mask = PreprocessLayer()((input_1, input_2)) # output shape (batch_size, 15, 3)
embedded_inputs_pos = AgentPositionEncoder(output_shape=hidden_size-8, embedding_size=hidden_size)(masked_inputs)
embedded_inputs_pose = AgentKeypointsEncoder(output_shape=hidden_size-8, embedding_size=hidden_size)(masked_inputs)
embedded_inputs_temp = AgentTemporalEncoder(output_shape=hidden_size-8, embedding_size=hidden_size, num_steps=input_length)(masked_inputs)

encoded_agent, scores = FeatureAttnAgentEncoderLearnedLayer(input_length=input_length)((masked_inputs, mask))
self_encoded_agent, scores2 = AgentSelfAlignmentLayer()((encoded_agent, mask))
transformed1, scores3 = SelfAttnTransformerLayer(mask=True)((self_encoded_agent, mask))
transformed2, scores4 = SelfAttnTransformerLayer(mask=True)((transformed1, mask))
transformed3, logits = MultimodalityInduction()((transformed2, mask))
transformed4, scores5 = SelfAttnTransformerLayer(mask=True, multimodality_induced=True)((transformed3, mask))
transformed5, scores6 = SelfAttnModeTransformerLayer()(transformed4)
transformed6, scores7 = SelfAttnTransformerLayer(mask=True, multimodality_induced=True)((transformed5, mask))
transformed7, scores8 = SelfAttnModeTransformerLayer()(transformed6)
pred = Prediction2DPositionHeadLayer()(transformed7)


model1 = tf.keras.Model(inputs=(input_1, input_2), outputs=masked_inputs)
model2 = tf.keras.Model(inputs=(input_1, input_2), outputs=pred)

for (batch_x1, batch_x2) in train_dataset.take(1):
    #output = model1.predict((batch_x1, batch_x2))
    output2 = model2.predict((batch_x1, batch_x2))
    break



(15,)
(None, 15, 5, 128)
(15,)
(32, 15, 5, 128)
1/1 [==============================] - 1s 1s/step


In [37]:
output2.shape

(32, 15, 5, 5)